# PA1 Part A: Byte Pair Encoding (BPE)

## Overview
In this assignment, you will implement the **Byte Pair Encoding (BPE)** algorithm **from scratch** using Python.  
BPE is a widely used **subword tokenization algorithm** that forms the foundation of tokenizers used in modern language models such as GPT and BERT.

Rather than relying on existing tokenizer libraries, you will build the algorithm step by step to understand:
- How subword vocabularies are learned
- How frequent character patterns are merged
- How unseen or rare words are handled during tokenization

By the end of this assignment, you will have a fully working BPE tokenizer that can be trained on a corpus and applied to new text.

---

## What is Byte Pair Encoding?
Byte Pair Encoding is an **unsupervised, frequency-based tokenization algorithm**.  
It starts with individual characters as tokens and repeatedly merges the **most frequent adjacent token pairs** in the corpus. Over time, this process builds a vocabulary of meaningful subword units.

BPE allows language models to:
- Limit vocabulary size
- Handle rare and unseen words
- Capture common prefixes, suffixes, and roots

---

## Additional Reading
If you would like a more detailed explanation of the algorithm and worked examples, refer to the following resource:

🔗 **GeeksforGeeks – Byte Pair Encoding (BPE) in NLP**  
https://www.geeksforgeeks.org/nlp/byte-pair-encoding-bpe-in-nlp/



<div style="color:red">

## Rules & Constraints

Please read the following rules carefully before starting the assignment.

### ❌ Disallowed
- You **may NOT** use any existing tokenization libraries  
  (e.g., HuggingFace tokenizers, SentencePiece, GPT tokenizers, etc.)
- You **may NOT** use pre-implemented BPE code from external sources

### ✅ Allowed
- Standard Python libraries such as:
  - `collections`
  - `re`
  - `math`
- Pure Python data structures (lists, dictionaries, tuples)

### 📌 Implementation Requirements
- All parts of the **BPE algorithm must be implemented from scratch**
- You must follow the function signatures provided in the notebook
- Your code should be **clear, modular, and well-commented**
- Do **not** modify the provided test cells (if any)

### ⚠️ Academic Integrity
- You must write and submit **your own code**
- Copying implementations from online sources will be considered plagiarism


## Instructions
- <font color="red">Proceed cell by cell and complete all sections where you are asked to write code.</font>
- <font color="red">Carefully read the course outline for the plagiarism policy and late-day rules.</font>
- <font color="red">Run all cells before submitting to receive full credit.</font>
- <font color="red">Do not delete or modify any pre-written code.</font>
- <font color="red">Attempt every part; each section builds toward a complete understanding.</font>
Failure to follow these rules may result in marks deduction.

## Submission Instructions
- <font color="red">Rename the notebook according to you Student ID. For example if you student ID is 27010001, rename the file to 27010001_PART_A</font>
- <font color="red">Once you're done with both Part A and Part B, zip the folder containing both notebooks and rename the folder to your Student ID as well.</font>

</div>


## 1. Dataset: *Alice’s Adventures in Wonderland* [3 Points]

In this assignment, all students will train their Byte Pair Encoding (BPE) tokenizer on the **same fixed corpus**

This text is in the **public domain** and is commonly used in NLP experiments due to its clean language and rich word structure.


### Corpus Details
- Source: *Alice’s Adventures in Wonderland*
- Size: **500 words**
- Preprocessing:
  - Lowercased
  - Punctuation removed
  - Words split on whitespace
- Each word is treated as an **independent unit**



In [10]:
import os
print(os.getcwd())
print(os.listdir())

/content
['.config', 'alice.txt', 'sample_data']


In [11]:
from collections import Counter
import re

# Load the fixed corpus
with open("/content/alice.txt", "r", encoding="utf-8") as f:
    text = f.read().lower()

# Basic preprocessing
text = re.sub(r"[^a-z\s]", "", text)
words = text.split()

# Count word frequencies
corpus = Counter(words)

print("Number of unique words:", len(corpus))
print("Most common words:", corpus.most_common(10))

Number of unique words: 2738
Most common words: [('the', 1623), ('and', 842), ('to', 721), ('a', 622), ('she', 537), ('it', 526), ('of', 507), ('said', 462), ('i', 399), ('alice', 385)]


In [12]:
def build_initial_vocab(corpus):
    """
    Build the initial BPE vocabulary from a word-frequency corpus.

    Args:
        corpus (dict): A dictionary mapping words to their frequencies.
                       Example: {"low": 5, "lower": 2}

    Returns:
        vocab (dict): A dictionary mapping space-separated character sequences
                      (with </w> appended) to their frequencies.
    """
    vocab = {}

    # 1. Iterate over each word and its frequency in the corpus
    for word, freq in corpus.items():
        # 2. Convert the word into a list of characters
        chars = list(word)

        # 3. Append the end-of-word marker '</w>'
        chars.append("</w>")

        # 4. Join the symbols with spaces to form the vocabulary key
        key = " ".join(chars)

        # 5. Store the frequency in the vocab dictionary
        vocab[key] = freq

    return vocab

In [13]:
# DO NOT MODIFY THIS CODE, IT IS FOR GRADING.
vocab = build_initial_vocab(corpus)

print("Number of vocabulary entries:", len(vocab))

# Print a few example entries
for i, (k, v) in enumerate(vocab.items()):
    print(f"{k} -> {v}")
    if i == 4:
        break


Number of vocabulary entries: 2738
a l i c e </w> -> 385
w a s </w> -> 357
b e g i n n i n g </w> -> 13
t o </w> -> 721
g e t </w> -> 46


## 2. Computing Pair Frequencies [5 Points]

The next step in Byte Pair Encoding (BPE) is to compute the **frequency of adjacent symbol pairs** across the entire vocabulary.

At each iteration of BPE training:
- We identify the **most frequent adjacent pair of symbols**
- This pair will later be merged into a single new symbol

---

### What is a Symbol Pair?

- Given a vocabulary entry like:
- l o w </w>
- The adjacent symbol pairs are:
- (l, o), (o, w), (w, </w>)



If the word occurs with frequency `f`, then **each of these pairs contributes `f`** to the overall pair frequency count.

---

### Why Frequencies Matter

BPE is a **frequency-based greedy algorithm**:
- Pairs that occur most frequently are merged first
- Frequencies must be weighted by word counts in the corpus

Failing to account for frequencies correctly will result in incorrect merge behavior.

---

### Your Task

You will implement a function that:
1. Iterates over all vocabulary entries
2. Extracts adjacent symbol pairs
3. Accumulates their frequencies



In [14]:
def get_pair_frequencies(vocab):
    """
    Compute frequencies of adjacent symbol pairs in the vocabulary.

    Args:
        vocab (dict): A dictionary mapping space-separated symbol sequences
                      to their frequencies.

    Returns:
        pair_freqs (dict): A dictionary mapping symbol pairs (tuples)
                           to their total frequency.
    """
    pair_freqs = {}

    # 1. Iterate over each vocabulary entry and its frequency
    for entry, freq in vocab.items():
        # 2. Split the entry into a list of symbols
        symbols = entry.split()

        # 3. Extract all adjacent symbol pairs
        for i in range(len(symbols) - 1):
            pair = (symbols[i], symbols[i + 1])

            # 4. Accumulate pair frequencies weighted by word frequency
            pair_freqs[pair] = pair_freqs.get(pair, 0) + freq

    return pair_freqs

In [15]:
# DO NOT MODIFY THIS CODE, IT IS FOR GRADING.
pair_freqs = get_pair_frequencies(vocab)

# Print the most frequent pairs
sorted_pairs = sorted(pair_freqs.items(), key=lambda x: x[1], reverse=True)

print("Top 10 most frequent symbol pairs:")
for pair, freq in sorted_pairs[:10]:
    print(pair, "->", freq)


Top 10 most frequent symbol pairs:
('e', '</w>') -> 5716
('h', 'e') -> 3772
('t', 'h') -> 3476
('t', '</w>') -> 3265
('d', '</w>') -> 3202
('s', '</w>') -> 2202
('i', 'n') -> 2024
('e', 'r') -> 1807
('n', '</w>') -> 1786
('a', 'n') -> 1604


## 3. Merging the Most Frequent Pair [5 Points]

Once we have computed the frequencies of all adjacent symbol pairs, the next step in Byte Pair Encoding (BPE) is to **merge the most frequent pair**.

This merge:
- Replaces every occurrence of the selected symbol pair
- Creates a new combined symbol
- Updates the vocabulary accordingly

This operation is applied **globally** across the entire vocabulary.

---

### Example

Suppose the most frequent pair is: ('l', 'o')
- Then the symbols: l o w </w>
- become: lo w </w>

Importantly:
- Only **adjacent** occurrences are merged
- Other symbols remain unchanged
- Frequencies of words remain the same

### Your Task

You will implement a function that:
1. Takes a symbol pair to merge
2. Replaces all occurrences of that pair in the vocabulary
3. Returns an updated vocabulary



In [16]:
def merge_vocab(pair, vocab):
    """
    Merge all occurrences of a symbol pair in the vocabulary.

    Args:
        pair (tuple): A tuple of two symbols to merge, e.g. ('l', 'o')
        vocab (dict): Current BPE vocabulary mapping symbol sequences
                      to their frequencies.

    Returns:
        new_vocab (dict): Updated vocabulary after merging the pair.
    """
    new_vocab = {}
    a, b = pair
    merged_symbol = a + b

    # 1. Iterate over each vocabulary entry and its frequency
    for entry, freq in vocab.items():
        # 2. Split the entry into a list of symbols
        symbols = entry.split()

        new_symbols = []
        i = 0

        # 3. Scan through the symbols and merge occurrences of the given pair
        while i < len(symbols):
            if (
                i < len(symbols) - 1
                and symbols[i] == a
                and symbols[i + 1] == b
            ):
                # merge the pair
                new_symbols.append(merged_symbol)
                i += 2
            else:
                new_symbols.append(symbols[i])
                i += 1

        # 4. Reconstruct the updated symbol sequence
        new_entry = " ".join(new_symbols)

        # 5. Store it in new_vocab with the same frequency
        new_vocab[new_entry] = new_vocab.get(new_entry, 0) + freq

    return new_vocab

In [17]:
# DO NOT MODIFY THIS CODE, IT IS FOR GRADING.

# Identify the most frequent pair
pair_freqs = get_pair_frequencies(vocab)
best_pair = max(pair_freqs, key=pair_freqs.get)

print("Most frequent pair:", best_pair)

# Apply merge
vocab = merge_vocab(best_pair, vocab)

# Inspect updated vocabulary
for i, (k, v) in enumerate(vocab.items()):
    print(f"{k} -> {v}")
    if i == 4:
        break


Most frequent pair: ('e', '</w>')
a l i c e</w> -> 385
w a s </w> -> 357
b e g i n n i n g </w> -> 13
t o </w> -> 721
g e t </w> -> 46


## 4. Training the BPE Tokenizer [15 Points]

So far, you have implemented the individual components of Byte Pair Encoding (BPE):
- Initial vocabulary construction
- Pair frequency computation
- Merging a symbol pair

In this section, you will **combine all of these steps** into a full BPE training loop that performs **multiple merge operations**.

This loop is the core of the BPE algorithm.

---

### How BPE Training Works (Recap)

For a fixed number of merge operations:
1. Compute pair frequencies from the current vocabulary
2. Select the **most frequent symbol pair**
3. Merge that pair across the entire vocabulary
4. Record the merge operation
5. Repeat

Each merge introduces a new subword token into the vocabulary.

---

### Your Task

You will implement a function that:
- Trains a BPE tokenizer for a specified number of merges
- Returns both:
  - The final vocabulary
  - The ordered list of merge operations

The **order of merges matters** and must be preserved.

---
Important Notes

If no pairs are available to merge, you may stop early

In case of ties, Python’s default behavior is acceptable

Do not shuffle or reorder merges

In [18]:

def train_bpe(corpus, num_merges):
    """
    Train a Byte Pair Encoding (BPE) tokenizer.

    Args:
        corpus (dict): Word-frequency corpus.
        num_merges (int): Number of BPE merge operations to perform.

    Returns:
        vocab (dict): Final BPE vocabulary after all merges.
        merges (list): List of merged symbol pairs, in order.
    """
    # Step 1: Build the initial vocabulary
    vocab = build_initial_vocab(corpus)

    merges = []

    # Step 2: Perform BPE merges
    for _ in range(num_merges):
        # 1. Compute pair frequencies
        pair_freqs = get_pair_frequencies(vocab)

        # Stop early if no pairs remain
        if not pair_freqs:
            break

        # 2. Find the most frequent symbol pair
        best_pair = max(pair_freqs, key=pair_freqs.get)

        # 3. Merge that pair in the vocabulary
        vocab = merge_vocab(best_pair, vocab)

        # 4. Record the merge operation
        merges.append(best_pair)

        # 5. Updated vocab is used in next iteration automatically

    return vocab, merges

In [19]:
# DO NOT MODIFY THIS CODE, IT IS FOR GRADING.

toy = {"low": 5, "lower": 2, "newest": 6, "widest": 3}
vocab, merges = train_bpe(toy, num_merges=10)

print("Merges:", merges[:5])
print("Sample vocab:")
for i, (k, v) in enumerate(vocab.items()):
    print(k, "->", v)
    if i == 4:
        break


Merges: [('e', 's'), ('es', 't'), ('est', '</w>'), ('l', 'o'), ('lo', 'w')]
Sample vocab:
low</w> -> 5
low e r </w> -> 2
newest</w> -> 6
wi d est</w> -> 3


## Theoretical Question: Interpreting Early BPE Merges [4 Points]

Using the output of your BPE training:

### Question
Identify **one** of the **first 10 merge operations** learned by your BPE algorithm.

Your answer must include:

1. **The merge operation**  
   - Write the merged token explicitly

2. **Explanation of frequency**  
   - Explain *why* this merge is frequent in the given corpus  

3. **Corpus evidence**  
   - Reference **at least two words** from the corpus in which this merge appears  

### Notes
- You may use intermediate printouts from your code to support your explanation
- Answers that only list the merge without explanation will not receive full credit
- Be concise





**Merge operation:**  
('l', 'o') =`lo`

**Why this merge is frequent:**  
The letters **l** and **o** often appear next to each other in the text</b>
Because BPE always merges the most common adjacent letters first, this pair is learned early

**Corpus evidence:**  
This merged token appears in multiple words from the corpus, such as:
- **low**
- **lower**
- **lowest**

For example, `l o w </w>` becomes `lo w </w>` after the merge.


The merge `('l', 'o')=lo` is learned early because it occurs many times in common words, making it a useful and frequent pattern in the corpus.

## 5. Tokenizing New Text with Learned BPE Merges [4 Points]

Training BPE produces an **ordered list of merge operations**.  
To tokenize new text, we apply these merges (in order) to each word.

This simulates how BPE tokenizers are used in practice:
- Start with characters + end-of-word marker
- Apply merges sequentially
- Output the resulting subword tokens

---

### Key Idea: Merge Order Matters

If the merge operations are:
- ('l', 'o') -> 'lo'
- ('lo', 'w') -> 'low'


Then tokenizing `"low"` must apply these merges in the same order to recover:


---

### Your Task

You will implement a function that tokenizes a single word using the learned merge rules.

Steps:
1. Convert the word into a list of characters
2. Append `</w>`
3. For each learned merge pair:
   - Replace all adjacent occurrences of that pair with the merged symbol
4. Return the final list of symbols



In [20]:
def tokenize_word_bpe(word, merges):
    """
    Tokenize a single word using a learned list of BPE merges.

    Args:
        word (str): Input word (already lowercased, no punctuation).
        merges (list): List of merge operations (tuples), in order.

    Returns:
        tokens (list): List of BPE tokens after applying merges.
    """
    # Step 1: start from characters + end-of-word marker
    tokens = list(word) + ["</w>"]

    # Step 2: apply merges in order
    for a, b in merges:
        new_tokens = []
        i = 0

        while i < len(tokens):
            # If we find the merge pair (a, b), merge them
            if i < len(tokens) - 1 and tokens[i] == a and tokens[i + 1] == b:
                new_tokens.append(a + b)
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1

        tokens = new_tokens

    return tokens


In [21]:
# DO NOT MODIFY THIS CODE, IT IS FOR GRADING.

# Train a small BPE model
vocab, merges = train_bpe(corpus, num_merges=50)

test_words = ["alice", "wonderland", "rabbit", "curious", "unseenword"]

for w in test_words:
    print(w, "->", tokenize_word_bpe(w, merges))

# print(merges) -> maybe this will help with the theoratical questions. hmmmm

alice -> ['alice</w>']
wonderland -> ['w', 'on', 'd', 'er', 'l', 'and</w>']
rabbit -> ['r', 'a', 'b', 'b', 'it</w>']
curious -> ['c', 'u', 'r', 'i', 'ou', 's</w>']
unseenword -> ['u', 'n', 's', 'e', 'en', 'w', 'or', 'd</w>']


## Theoretical Question: Frequent but Linguistically Unintuitive Merges [5 Points]

### Question
Choose **one merge operation** from your learned BPE merges that is frequent but linguistically unintuitive.

Your answer must include:

1. **The merge operation**  
   - Write the merge explicitly

2. **Why BPE learns this merge**  
   - Explain why this merge has high frequency in the corpus  

3. **Impact on tokenization**  
   - State whether this merge **helps or hurts** downstream tokenization. Justify your answer (briefly)

### Notes
- The merge must be taken from your learned merge list
- Focus on reasoning rather than linguistic terminology
- Answers should be grounded in your observed merge statistics





**Merge operation:**  
`('e', '</w>')  e</w>`

**Why BPE learns this merge:**  
BPE learns this merge because it looks only at how often letter pairs appear together.  
In the corpus, many words end with the letter **e**, so the pair **e </w>** appears very often.

From the training output, this pair had the **highest frequency**, so BPE merged it early.

**Examples from the corpus:**  
This merge appears at the end of many common words, such as:
- **the**
- **she**
- **alice**

Each of these words ends with `e`, followed by the end-of-word marker `</w>`.

**Impact on tokenization:**  
This merge mostly **hurts or gives little benefit**.

**Reason:**  
- The token `e</w>` does not represent a meaningful part of a word.
- It is only frequent because of word endings, not because it carries meaning.
- It uses up a merge operation that could have been used for more useful patterns.

**Conclusion:**  
The merge `('e', '</w>')  e</w>` is frequent due to many words ending in `e`, but it is linguistically unintuitive and not very helpful for good tokenization.

## 6. Tokenizing Full Text with BPE [5 Points]

So far, you have implemented BPE tokenization for a **single word**.  
In practice, tokenizers are applied to **entire texts**, usually by tokenizing each word independently and then concatenating the results.

In this section, you will use your existing `tokenize_word_bpe` function to tokenize a full input string.

---

### Your Task

You will implement a helper function that:
1. Splits the input text on whitespace
2. Applies BPE tokenization to each word
3. Returns a flattened list of BPE tokens

This mirrors how BPE-based tokenizers are typically used in real NLP pipelines.


In [23]:
def tokenize_text_bpe(text, merges):
    """
    Tokenize a whitespace-separated string using BPE merges.

    Args:
        text (str): Input text (already lowercased/cleaned).
        merges (list): Learned BPE merges (tuples), in order.

    Returns:
        tokens (list): Flattened list of BPE tokens for the full text.
    """
    tokens = []

    # 1. Split text into words
    words = text.split()

    # 2. Tokenize each word and collect tokens
    for word in words:
        word_tokens = tokenize_word_bpe(word, merges)
        tokens.extend(word_tokens)

    return tokens

In [24]:
# DO NOT MODIFY THIS CODE, IT IS FOR GRADING.
sample = "alice was beginning to get very tired"
tokens = tokenize_text_bpe(sample.lower(), merges)

print(tokens)


['alice</w>', 'w', 'as</w>', 'b', 'e', 'g', 'in', 'n', 'ing</w>', 'to</w>', 'g', 'e', 't</w>', 'v', 'er', 'y</w>', 't', 'i', 'r', 'ed</w>']


## Theoretical Question: Limitations of BPE Tokenization [4 Points]

### Question
Provide **one example word** that is **poorly tokenized** after training your BPE tokenizer.

Your answer must include:

1. **Example word**  
   - The word must be taken from the Alice corpus

2. **Explanation**  
   - Explain which merge operations led to this tokenization  
   - Describe why the resulting tokenization may be considered suboptimal or undesirable


### Notes
- Answers must reference observed behavior from your tokenizer
- Generic answers will not receive full credit





**Example word:**  
`beginning`

**Observed tokenization:**  
`b e g in n ing</w>`

**Explanation:**  
During training, BPE learned very common letter patterns such as:
- `('i', 'n') in`
- `('i', 'n', 'g') ing</w>`

These patterns appear in many different words, so they were merged early  
However, the full word **beginning** is not very frequent in the corpus, so BPE did not learn merges that combine the whole word or larger parts like `begin`.

**Why this tokenization is suboptimal:**  
The word is split into many small pieces, which increases the number of tokens.  
This makes the tokenization less efficient and less intuitive compared to a split like `begin + ning</w>` or a single token.

**Conclusion:**  
This shows a limitation of BPE: it merges frequent patterns globally, but this can lead to poor tokenization for some individual words

## Generative AI Usage Report

This assignment was completed with the assistance of a generative AI tool, in accordance with the course AI policy.

### AI Tool Used
- **Tool:** ChatGPT 5.2 – Thinking  
- **Provider:** OpenAI  
- **Account Type:** Plus Account  

---

### Purpose of AI Use
The AI tool was used as a **learning and support aid**, not as a direct replacement for independent work. Specifically, it was used to:
- Understand the **Byte Pair Encoding (BPE)** algorithm conceptually
- Receive guidance on implementing BPE components from scratch in Python
- Clarify theoretical questions related to BPE behavior and limitations
- Improve clarity and structure of written theoretical explanations

---

### Prompts Used (Examples)
- “Explain what this BPE function does step by step.”
- “Help implement BPE merge and pair frequency functions from scratch.”
- “Explain why this BPE merge is frequent but linguistically unintuitive.”


---

### AI Outputs Used
- Explanations of BPE concepts (pair frequencies, merges, tokenization)
- Example implementations of BPE-related functions (adapted to match assignment constraints)
- Draft explanations for theoretical questions, later refined by the student

---

### Edits and Student Contributions
- All AI-generated code was **reviewed, tested, and adapted** by the student.
- Variable names, formatting, and logic were adjusted to match assignment requirements.
- Theoretical answers were **rewritten in simpler wording** and aligned with the student’s actual outputs.
- Final responses reflect the student’s understanding and interpretation of results.

---

### Academic Integrity Statement
The student confirms that:
- No prohibited libraries or external BPE implementations were used.
- All submitted work was reviewed and understood by the student.
- AI assistance was used transparently as a support tool, not to bypass learning objectives.

---

### Example Citation
ChatGPT 5.2 – Thinking (2026).  
Prompt: “Explain why this BPE merge is frequent but linguistically unintuitive.”  
Response: Explanation of frequency-based merges and their impact on tokenization.  
OpenAI. Plus Account.